# Words to Floor Plan (W2FP) Generative adversarial network (GAN)

#### Imports
Packages and libraries used through-out the project.

In [6]:
import os
import time

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model, layers, losses
from IPython import display

tf.__version__

'2.8.0'

#### Hyper-parametes

In [26]:
MAX_SEQUENCE_LENGTH=128
VOCAB_SIZE = 128

#### Text vectorization
Defining text conversion utility, which processes text to be appropriate for neural network input.

In [17]:
int_vectorize_layer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

#### Generator neural network

In [39]:
def create_generator_model() -> Model:
    model = tf.keras.Sequential()

    # `VOCAB_SIZE + 1` since `0` is used additionally for padding.
    model.add(layers.Embedding(VOCAB_SIZE + 1, 128, input_length=MAX_SEQUENCE_LENGTH, mask_zero=True)),
    model.add(layers.Conv1D(64, 5, padding="same", activation="relu", strides=2))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(8*8*64))

    model.add(layers.Reshape((8, 8, 64)))

    model.add(layers.Conv2DTranspose(512, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

In [40]:
generator = create_generator_model()
generator.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 128, 128)          16512     
                                                                 
 conv1d_6 (Conv1D)           (None, 64, 64)            41024     
                                                                 
 global_max_pooling1d_4 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 4096)              266240    
                                                                 
 reshape_10 (Reshape)        (None, 8, 8, 64)          0         
                                                                 
 conv2d_transpose_40 (Conv2D  (None, 8, 8, 512)        819200    
 Transpose)                                          

In [41]:
noise = int_vectorize_layer("gosho losho posho mosho")
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

2022-03-20 02:00:02.399548: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at lookup_table_op.cc:929 : FAILED_PRECONDITION: Table not initialized.


FailedPreconditionError: Exception encountered when calling layer "string_lookup" (type StringLookup).

Table not initialized. [Op:LookupTableFindV2]

Call arguments received:
  • inputs=tf.Tensor(shape=(4,), dtype=string)

#### Discriminator neural network

In [38]:
def create_discriminator_model() -> Model:
    model = tf.keras.Sequential()

    model.add(
        layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[64, 64, 1]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[64, 32, 32]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[128, 16, 16]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(512, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[256, 8, 8]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(512, (4, 4), strides=(1, 1), padding='same', use_bias=False, input_shape=[512, 4, 4]))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [40]:
discriminator = create_discriminator_model()
discriminator.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 32, 32, 64)        1024      
                                                                 
 batch_normalization_48 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_46 (LeakyReLU)  (None, 32, 32, 64)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 16, 128)       131072    
                                                                 
 batch_normalization_49 (Bat  (None, 16, 16, 128)      512       
 chNormalization)                                                
                                                                 
 leaky_re_lu_47 (LeakyReLU)  (None, 16, 16, 128)     